In [1]:
import mint
import numpy

In [2]:
regridder = mint.RegridEdges()

# grid settings
regridder.setSrcGridFlags(fixLonAcrossDateline=0, averageLonAtPole=0)
regridder.setDstGridFlags(fixLonAcrossDateline=1, averageLonAtPole=1)

# load the grids
regridder.loadSrcGrid('../../data/latlon100x50.nc$latlon')
regridder.loadDstGrid('../../data/lfric_diag_wind.nc$Mesh2d')

# compute the regridding weights
regridder.build(numCellsPerBucket=10, periodX=360., debug=1)

mint.printLogMessages()
mint.writeLogMessages('log.txt')

In [ ]:
!  cat log.txt